In [ ]:
import os
import numpy as np
import scipy.io
from mne import filter
from mne.time_frequency import psd_array_multitaper
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
# Define directories
audio_directory = 'E:/Research/EEG_Data/pre_iso-ano/Final_preposed_iso'
visual_directory = 'E:/Research/EEG_Data/pre_iso-ano/Final_preposed_ano'

# Helper function for band power calculation
def calc_band_power(psd_data, freqs, band):
    idx_band = np.logical_and(freqs >= band[0], freqs <= band[1])
    return np.mean(psd_data[:, idx_band], axis=1)

# Initialize variables to store cumulative power spectra and feature data
features = {
    'iso_theta': [], 'iso_alpha': [], 'iso_beta': [], 'iso_gamma': [], 'iso_delta': [],
    'ano_theta': [], 'ano_alpha': [], 'ano_beta': [], 'ano_gamma': [], 'ano_delta': []
}

def load_and_preprocess(directory, feature_key):
    cumulative_psd = None
    files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.mat')]
    
    for file in files:
        data = scipy.io.loadmat(file)['data']
        
        # Apply notch filter at 60 Hz
        data_filtered = filter.notch_filter(data, Fs=1000, freqs=60)
        
        # Compute PSD (Power Spectral Density)
        psd, freqs = psd_array_multitaper(data_filtered, sfreq=1000, fmin=0.1, fmax=100)
        
        # Accumulate PSD for averaging
        if cumulative_psd is None:
            cumulative_psd = psd
        else:
            cumulative_psd += psd
        
        # Extract band powers
        features[feature_key + '_theta'].append(calc_band_power(psd, freqs, [4, 8]))
        features[feature_key + '_alpha'].append(calc_band_power(psd, freqs, [8, 12]))
        features[feature_key + '_beta'].append(calc_band_power(psd, freqs, [13, 30]))
        features[feature_key + '_gamma'].append(calc_band_power(psd, freqs, [30, 100]))
        features[feature_key + '_delta'].append(calc_band_power(psd, freqs, [0.5, 4]))
    
    return cumulative_psd / len(files)

In [ ]:
# Process iso and ano data
cumulative_psd_iso = load_and_preprocess(audio_directory, 'iso')
cumulative_psd_ano = load_and_preprocess(visual_directory, 'ano')

# Convert features to numpy arrays
X = np.hstack([np.array(features[key]) for key in features.keys()])
y = np.array([0] * len(features['iso_theta']) + [1] * len(features['ano_theta']))  # Assuming 0 for iso, 1 for ano

# Split Data into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# SVM Model
svm_model = SVC(kernel='rbf', probability=True)
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)

In [ ]:
# Random Forest Model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

In [ ]:
# KNN Model
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)
y_pred_knn = knn_model.predict(X_test)

In [ ]:
# Gradient Boosting Machine (GBM) Model
gbm_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
gbm_model.fit(X_train, y_train)
y_pred_gbm = gbm_model.predict(X_test)

In [ ]:
# DNN Model using TensorFlow/Keras
dnn_model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

dnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
dnn_model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))

y_pred_dnn = np.argmax(dnn_model.predict(X_test), axis=1)

In [ ]:
# CNN Model using TensorFlow/Keras
X_train_cnn = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_cnn = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

cnn_model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(16, 3, padding='same', activation='relu', input_shape=(X_train.shape[1], 1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2, activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
cnn_model.fit(X_train_cnn, y_train, epochs=20, batch_size=64, validation_data=(X_test_cnn, y_test))

y_pred_cnn = np.argmax(cnn_model.predict(X_test_cnn), axis=1)

In [ ]:
# Accuracy Calculation and Confusion Matrix Plotting
models = {
    'SVM': y_pred_svm,
    'Random Forest': y_pred_rf,
    'KNN': y_pred_knn,
    'GBM': y_pred_gbm,
    'DNN': y_pred_dnn,
    'CNN': y_pred_cnn
}

for model_name, y_pred in models.items():
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy of {model_name}: {accuracy * 100:.2f}%')
    ConfusionMatrixDisplay.from_predictions(y_test, y_pred)
    plt.title(f'Confusion Matrix for {model_name}')
    plt.show()

In [ ]:
# Plot Model Accuracy Comparison
accuracies = [accuracy_score(y_test, y_pred) for y_pred in models.values()]
plt.figure()
plt.bar(models.keys(), [acc * 100 for acc in accuracies])
plt.ylabel('Accuracy (%)')
plt.title('Model Accuracy Comparison')
plt.show()
